In [3]:
import torch

batch_size = 1
seq_len = 3
input_size = 4
hidden_size = 2
num_layers = 1

cell = torch.nn.RNN(input_size = input_size,hidden_size = hidden_size,
                   num_layers = num_layers,batch_first = True)

inputs = torch.randn(batch_size,seq_len,input_size)
hidden = torch.zeros(num_layers,batch_size,hidden_size)

out,hidden = cell(inputs,hidden)

print('outpue size:', out.shape)
print('outputL',out)
print('hidden size:',hidden.shape)
print('hidden:',hidden)

outpue size: torch.Size([1, 3, 2])
outputL tensor([[[ 0.7260, -0.9006],
         [ 0.6355, -0.3579],
         [ 0.7950, -0.4178]]], grad_fn=<TransposeBackward1>)
hidden size: torch.Size([1, 1, 2])
hidden: tensor([[[ 0.7950, -0.4178]]], grad_fn=<StackBackward0>)


# 使用RNNCell

In [11]:
import torch

input_size = 4
hidden_size = 4
batch_size = 1

idx2char = ['e','h','l','o']
x_data = [1,0,2,2,3]
y_data = [3,1,2,3,2]

one_hot_lookup = [[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,0],
                 [0,0,0,1]]

x_one_hot = [one_hot_lookup[x] for x in x_data]
print('x_one_hot:',x_one_hot)

inputs = torch.Tensor(x_one_hot).view(-1,batch_size,input_size)
labels = torch.LongTensor(y_data).view(-1,1)

class Model(torch.nn.Module):
    def __init__(self,input_size,hidden_size,batch_size):
        super(Model,self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnncell = torch.nn.RNNCell(input_size = self.input_size,
                                       hidden_size = self.hidden_size)
        
    def forward(self,input,hidden):
            hidden = self.rnncell(input,hidden)
            return hidden
        
    def init_hidden(self):
            return torch.zeros(self.batch_size,self.hidden_size)
        
net = Model(input_size,hidden_size,batch_size)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr = 0.1)

for epoch in range(20):
    loss = 0
    optimizer.zero_grad()
    hidden = net.init_hidden()
    print('Prediect String:', end = '')
    for input , label in zip(inputs,labels):
        hidden = net(input,hidden)
        loss += criterion(hidden,label)
        _,idx = hidden.max(dim=1)
        print(idx2char[idx.item()],end='')        
    loss.backward()
    optimizer.step()
    print(',Epoch [%d/20] loss=%.4f' % (epoch+1 , loss.item()))

x_one_hot: [[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
Prediect String:heehh,Epoch [1/20] loss=7.6047
Prediect String:hhhhl,Epoch [2/20] loss=6.3010
Prediect String:hhlll,Epoch [3/20] loss=5.5364
Prediect String:hhlll,Epoch [4/20] loss=4.9804
Prediect String:ohlll,Epoch [5/20] loss=4.5443
Prediect String:ohlll,Epoch [6/20] loss=4.2089
Prediect String:ohlll,Epoch [7/20] loss=3.9519
Prediect String:ohlol,Epoch [8/20] loss=3.7429
Prediect String:ohlol,Epoch [9/20] loss=3.5568
Prediect String:ohlol,Epoch [10/20] loss=3.3820
Prediect String:ohlol,Epoch [11/20] loss=3.2161
Prediect String:ohlol,Epoch [12/20] loss=3.0521
Prediect String:ohlol,Epoch [13/20] loss=2.8744
Prediect String:ohlol,Epoch [14/20] loss=2.6806
Prediect String:ohlol,Epoch [15/20] loss=2.4981
Prediect String:ohlol,Epoch [16/20] loss=2.3659
Prediect String:ohlol,Epoch [17/20] loss=2.2986
Prediect String:ohlol,Epoch [18/20] loss=2.2496
Prediect String:ohlol,Epoch [19/20] loss=2.1780
Prediect String

# 使用RNN

In [15]:
import torch

input_size = 4
hidden_size = 4
num_size = 1
batch_size = 1
seq_len = 5

idx2char = ['e','h','l','o']
x_data = [1,0,2,2,3]
y_data = [3,1,2,3,2]

one_hot_lookup = [[1,0,0,0],
                 [0,1,0,0],
                 [0,0,1,0],
                 [0,0,0,1]]

x_one_hot = [one_hot_lookup[x] for x in x_data]
print('x_one_hot:',x_one_hot)

inputs = torch.Tensor(x_one_hot).view(seq_len,batch_size,input_size)
labels = torch.LongTensor(y_data)

class Model(torch.nn.Module):
    def __init__(self,input_size,hidden_size,batch_size,num_layers=1):
        super(Model,self).__init__()
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = torch.nn.RNN(input_size = self.input_size,
                                    hidden_size = self.hidden_size,
                                    num_layers = self.num_layers)
        
    def forward(self,input):
            hidden = torch.zeros(self.num_layers,self.batch_size,self.hidden_size)
            out,_ = self.rnn(input,hidden)
            return out.view(-1,self.hidden_size)
        
net = Model(input_size,hidden_size,batch_size,num_layers)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr = 0.05)

for epoch in range(20):
    optimizer.zero_grad()
    outputs = net(inputs)
    loss = criterion(outputs,labels)
    loss.backward()   
    optimizer.step()
    
    _,idx = hidden.max(dim=1)
    idx = idx.data.numpy()
    print('Prediect String:', ''.join([idx2char[x] for x in idx]),end = '')
    print(',Epoch [%d/20] loss=%.3f' % (epoch+1 , loss.item()))

x_one_hot: [[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 0, 1]]
Prediect String: l,Epoch [1/20] loss=1.203
Prediect String: l,Epoch [2/20] loss=1.086
Prediect String: l,Epoch [3/20] loss=0.996
Prediect String: l,Epoch [4/20] loss=0.922
Prediect String: l,Epoch [5/20] loss=0.858
Prediect String: l,Epoch [6/20] loss=0.798
Prediect String: l,Epoch [7/20] loss=0.739
Prediect String: l,Epoch [8/20] loss=0.683
Prediect String: l,Epoch [9/20] loss=0.633
Prediect String: l,Epoch [10/20] loss=0.593
Prediect String: l,Epoch [11/20] loss=0.561
Prediect String: l,Epoch [12/20] loss=0.535
Prediect String: l,Epoch [13/20] loss=0.514
Prediect String: l,Epoch [14/20] loss=0.497
Prediect String: l,Epoch [15/20] loss=0.481
Prediect String: l,Epoch [16/20] loss=0.466
Prediect String: l,Epoch [17/20] loss=0.452
Prediect String: l,Epoch [18/20] loss=0.439
Prediect String: l,Epoch [19/20] loss=0.427
Prediect String: l,Epoch [20/20] loss=0.417
